<a href="https://colab.research.google.com/github/bs3537/DS-Unit-4-Sprint-2-Neural-Networks/blob/master/_Learning_rate_%2Bepochs_tuning__Keras_GridsearchCV_Hyperparameter_Churn_Tuning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [0]:
import numpy as np

import matplotlib.pyplot as plt

import pandas as pd

In [0]:
df = pd.read_csv('https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv')

In [0]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [0]:
df.shape

(7043, 21)

In [0]:
df['customerID'].nunique()

7043

In [0]:
df = df.drop(columns="customerID")

In [0]:
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [0]:
df['Churn'] = df['Churn'].replace(regex='Yes', value=1)
df['Churn'] = df['Churn'].replace(regex='No', value=0)

In [0]:
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,0
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,0
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,0
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,1


In [0]:
#Split into train and validation datasets

from sklearn.model_selection import train_test_split
train, val = train_test_split(df, train_size=0.80, test_size=0.20, stratify=df['Churn'], random_state=42)
train.shape, val.shape

((5634, 20), (1409, 20))

In [0]:
train['Churn'].value_counts(normalize=True)

0    0.734647
1    0.265353
Name: Churn, dtype: float64

In [0]:
val['Churn'].value_counts(normalize=True)

0    0.734564
1    0.265436
Name: Churn, dtype: float64

In [0]:
#Thus, we have equal distribution of Churn_Yes in both train and val. and both datasets are balanced.

####Baseline target (churn): Zero features in train dataset

In [0]:
# Train dataset baseline
target = 'Churn'
y_train = train[target]
y_train.value_counts(normalize=True)

0    0.734647
1    0.265353
Name: Churn, dtype: float64

In [0]:
majority_class = y_train.mode()[0]
y_pred = [majority_class] * len(y_train)


In [0]:
#baseline accuracy if we guessed the majority class for every prediction on train dataset

from sklearn.metrics import accuracy_score
accuracy_score(y_train, y_pred)

#Baseline accuracy with no features = 73.46%

0.7346467873624423

In [0]:
# 3. Arrange X feature matrices (already did y target vectors)
target = 'Churn'
X_train = train.drop(columns=target)
y_train = train[target]
X_val = val.drop(columns=target)
y_val = val[target]

In [0]:
#Encoding

from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()

X_train_encoded = encoder.fit_transform(X_train)

X_val_encoded = encoder.fit_transform(X_val)

In [0]:
# Scale the data before doing logistic regression
from sklearn.preprocessing import Normalizer
norm = Normalizer()
X_train_norm = norm.fit_transform(X_train_encoded)
X_val_norm = norm.transform(X_val_encoded)

####Using Keras Dense Feed forward NN for classification with GridsearchCV

In [0]:
import tensorflow

from tensorflow import keras

In [0]:
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense, Dropout, PReLU

from tensorflow.keras.callbacks import EarlyStopping

import numpy as np

##Hyperparameter tuning using GridSearch

In [0]:
from sklearn.model_selection import GridSearchCV

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from tensorflow.keras.optimizers import Adam, Nadam

In [0]:
X_train_norm.shape[1:]

(19,)

In [0]:
inputs = X_train_norm.shape[1]

In [0]:
inputs

19

####GridSearch CV for learning rate

In [0]:
def create_model(learn_rate=0.001):
    # create model

    model = Sequential()

    model.add(Dense(128, input_shape=(inputs,)))
    model.add(tensorflow.keras.layers.PReLU())
    #model.add(Dropout(0.2)) 

    model.add(Dense(64)) 
    model.add(tensorflow.keras.layers.PReLU())  #Hidden Layer 1
    #model.add(Dropout(0.2)) 

    model.add(Dense(32)) 
    model.add(tensorflow.keras.layers.PReLU())  #Hidden Layer 1
    #model.add(Dropout(0.2)) 


    model.add(Dense(1, activation='sigmoid'))


    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
kf_model = KerasClassifier(build_fn=create_model, verbose=0)


In [0]:
# Define the grid search parameters
#learn_rate= [0.0005, 0.001, 0.0015, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01] 

#epochs: [20]

#optimizer = ['adam', 'nadam']
#batch_size = [0, 10, 20, 30]
#epochs = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
#dropout_rate = [0.0, 0.2, 0.3, 0.4, 0.5]
#neurons = [32, 64, 128] #hidden layer

#params = {'batch_size': [10, 50, 100, 250, 500, 1000, 2500], 'epochs': [20]}

params = {'learn_rate': [0.0005, 0.001, 0.0015, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01] , 'epochs': [20]}


In [0]:
# Create Grid Search for learning rate


grid = GridSearchCV(estimator=kf_model, param_grid=params, n_jobs=-1, verbose=1, scoring='accuracy')
grid_result = grid.fit(X_train_norm, y_train)

print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}')

means = grid_result.cv_results_['mean_test_score']

stds = grid_result.cv_results_['std_test_score']

params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print(f'Mean: {mean}, Stdev: {stdev} with : {param}')

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 out of  60 | elapsed:   36.8s remaining:   15.8s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   37.3s finished


Best: 0.7910898485581582 using {'epochs': 20, 'learn_rate': 0.002}
Mean: 0.7806162638041547, Stdev: 0.012249030157011911 with : {'epochs': 20, 'learn_rate': 0.0005}
Mean: 0.7827456536711526, Stdev: 0.013199250739087549 with : {'epochs': 20, 'learn_rate': 0.001}
Mean: 0.7797250122537237, Stdev: 0.01934754354456348 with : {'epochs': 20, 'learn_rate': 0.0015}
Mean: 0.7910898485581582, Stdev: 0.0067466938412139065 with : {'epochs': 20, 'learn_rate': 0.002}
Mean: 0.7850550274940464, Stdev: 0.012578252306920357 with : {'epochs': 20, 'learn_rate': 0.003}
Mean: 0.7751083134620749, Stdev: 0.02605392743961833 with : {'epochs': 20, 'learn_rate': 0.004}
Mean: 0.7811495962969326, Stdev: 0.010231045006434914 with : {'epochs': 20, 'learn_rate': 0.005}
Mean: 0.7841650367769317, Stdev: 0.013940173328809806 with : {'epochs': 20, 'learn_rate': 0.006}
Mean: 0.781858657433164, Stdev: 0.013740486846911384 with : {'epochs': 20, 'learn_rate': 0.007}
Mean: 0.7873617220461433, Stdev: 0.008353409273842278 with :

In [0]:
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

Best: 0.7910898485581582 using {'epochs': 20, 'learn_rate': 0.002}


In [0]:
pipeline1 = grid_result.best_estimator_

In [0]:
y_pred = pipeline1.predict(X_val_norm)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [0]:
from sklearn.metrics import accuracy_score
val_accuracy = accuracy_score(y_val, y_pred)
print("Gridsearch Keras hypertuning validation accuracy=", val_accuracy)

Gridsearch Keras hypertuning validation accuracy= 0.78708303761533


In [0]:
y_pred2 = (pipeline1.predict(X_val_norm) > 0.5).astype("int32")

In [0]:
val_accuracy = accuracy_score(y_val, y_pred2, normalize=True)
print("Gridsearch Keras hypertuning validation accuracy=", val_accuracy)

Gridsearch Keras hypertuning validation accuracy= 0.78708303761533


##Tune epochs 

###Gridsearch run #2

In [0]:
def create_model(epochs=10, learn_rate=0.002):
    # create model

    model = Sequential()

    model.add(Dense(128, input_shape=(inputs,)))
    model.add(tensorflow.keras.layers.PReLU())
    #model.add(Dropout(0.2)) 

    model.add(Dense(64)) 
    model.add(tensorflow.keras.layers.PReLU())  #Hidden Layer 1
    #model.add(Dropout(0.2)) 

    model.add(Dense(32)) 
    model.add(tensorflow.keras.layers.PReLU())  #Hidden Layer 1
    #model.add(Dropout(0.2)) 


    model.add(Dense(1, activation='sigmoid'))


    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
kf_model2 = KerasClassifier(build_fn=create_model, verbose=0)

In [0]:
params2 = {'learn_rate': [0.002] , 'epochs': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}

In [0]:

grid2 = GridSearchCV(estimator=kf_model2, param_grid=params2, n_jobs=-1, verbose=1, scoring='accuracy')
grid_result2 = grid2.fit(X_train_norm, y_train)

print(f'Best: {grid_result2.best_score_} using {grid_result2.best_params_}')

means = grid_result2.cv_results_['mean_test_score']

stds = grid_result2.cv_results_['std_test_score']

params = grid_result2.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print(f'Mean: {mean}, Stdev: {stdev} with : {param}')

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done  22 out of  50 | elapsed:   10.8s remaining:   13.8s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   15.7s finished


Best: 0.7767100445862181 using {'epochs': 10, 'learn_rate': 0.002}
Mean: 0.7358896203473282, Stdev: 0.00909996239623131 with : {'epochs': 1, 'learn_rate': 0.002}
Mean: 0.7342930901606144, Stdev: 0.010354675117440995 with : {'epochs': 2, 'learn_rate': 0.002}
Mean: 0.7406828358032532, Stdev: 0.00524512124535678 with : {'epochs': 3, 'learn_rate': 0.002}
Mean: 0.7383742500011821, Stdev: 0.007820801313398075 with : {'epochs': 4, 'learn_rate': 0.002}
Mean: 0.7444112775235974, Stdev: 0.009463695439637348 with : {'epochs': 5, 'learn_rate': 0.002}
Mean: 0.7511548445156114, Stdev: 0.02121271536787617 with : {'epochs': 6, 'learn_rate': 0.002}
Mean: 0.760916216050093, Stdev: 0.02145058416006904 with : {'epochs': 7, 'learn_rate': 0.002}
Mean: 0.7754687541863606, Stdev: 0.012727368775685253 with : {'epochs': 8, 'learn_rate': 0.002}
Mean: 0.7655325996334128, Stdev: 0.022612969260103533 with : {'epochs': 9, 'learn_rate': 0.002}
Mean: 0.7767100445862181, Stdev: 0.016018659540029533 with : {'epochs': 10

In [0]:
print(f"Best: {grid_result2.best_score_} using {grid_result2.best_params_}")

Best: 0.7767100445862181 using {'epochs': 10, 'learn_rate': 0.002}


In [0]:
pipeline2 = grid_result2.best_estimator_

In [0]:
y_pred = pipeline2.predict(X_val_norm)

In [0]:
from sklearn.metrics import accuracy_score
val_accuracy = accuracy_score(y_val, y_pred)
print("Gridsearch Keras hypertuning validation accuracy=", val_accuracy)

Gridsearch Keras hypertuning validation accuracy= 0.78708303761533
